# Gravity Model with PolityV Scores
---

### Basic Specification:
$$ y_{ijt} = \alpha + \beta_1 GDP_i + \beta_2 GDP_j + \beta_3 Dist_{ij} + \theta_t + \epsilon_{ijt} $$

### Modified Specification:
$$ y_{ijt} = \alpha + \beta_1 GDP_i + \beta_2 GDP_j + \beta_3 Dist_{ij} + \beta_4 PolityDist_{ijt} + \theta_t + \epsilon_{ijt} $$

In [1]:
import numpy as np
import pandas as pd
from linearmodels import PanelOLS

In [2]:
data = pd.read_csv("../data/trade_polity_full.csv")
data.head(10)

,tradelink,year,iso_o,iso_d,contig,comlang_off,distw,pop_o,gdp_o,gdpcap_o,...,family,polity_o,polity_d,polity_dist,log_gdp_o,log_gdp_d,log_flow,log_distw,log_polity_dist,log_col_hist
0,AFG-ARG,1952,AFG,ARG,0,0,15341.161651,8.425,NaN,NaN,...,NaN,-10,-9,1,NaN,8.942325,NaN,9.638360,0.693147,0.0
1,AUS-ARG,1952,AUS,ARG,0,0,12044.573996,8.691,8007.00,921.297900,...,NaN,10,-9,19,8.988071,8.942325,NaN,9.396453,2.995732,0.0
2,BEL-ARG,1952,BEL,ARG,0,0,11305.285880,8.730,8132.00,931.500600,...,NaN,9,-9,18,9.003562,8.942325,1.509918,9.333114,2.944439,0.0
3,BOL-ARG,1952,BOL,ARG,1,1,1865.839405,2.883,303.85,105.393690,...,ESP,4,-9,13,5.716534,8.942325,0.495826,7.532002,2.639057,0.0
4,BRA-ARG,1952,BRA,ARG,1,0,2391.846403,56.603,6211.00,109.729164,...,NaN,5,-9,14,8.734077,8.942325,1.746797,7.780239,2.708050,0.0
5,CAN-ARG,1952,CAN,ARG,0,0,9391.460934,14.786,25117.00,1698.701400,...,NaN,10,-9,19,10.131300,8.942325,1.257806,9.147663,2.995732,0.0
6,CHL-ARG,1952,CHL,ARG,1,1,1156.725891,6.378,2360.63,370.120730,...,ESP,2,-9,11,7.766684,8.942325,1.571842,7.054213,2.484907,0.0
7,CHN-ARG,1952,CHN,ARG,0,0,19110.127211,568.910,23457.00,41.231480,...,NaN,-8,-9,1,10.062924,8.942325,NaN,9.858026,0.693147,0.0
8,COL-ARG,1952,COL,ARG,0,1,4615.471326,12.351,3813.00,308.719940,...,ESP,-5,-9,4,8.246172,8.942325,0.799206,8.437386,1.609438,0.0
9,CUB-ARG,1952,CUB,ARG,0,1,6477.504793,6.008,2007.00,334.054600,...,ESP,-88,-9,79,7.604396,8.942325,NaN,8.776245,4.382027,0.0


In [3]:
data.columns

Index(['tradelink', 'year', 'iso_o', 'iso_d', 'contig', 'comlang_off', 'distw',
       'pop_o', 'gdp_o', 'gdpcap_o', 'iso2_o', 'pop_d', 'gdp_d', 'gdpcap_d',
       'iso2_d', 'heg_d', 'conflict', 'indepdate', 'heg_o', 'col_to', 'col_fr',
       'col_hist', 'col_cur', 'sever', 'sib_conflict', 'gatt_o', 'gatt_d',
       'rta', 'comleg', 'comcur', 'acp_to_eu', 'gsp', 'eu_to_acp', 'gsp_rec',
       'flow', 'validmirror', 'family', 'polity_o', 'polity_d', 'polity_dist',
       'log_gdp_o', 'log_gdp_d', 'log_flow', 'log_distw', 'log_polity_dist',
       'log_col_hist'],
      dtype='object')

In [4]:
data = data.set_index(['tradelink', 'year'])

In [5]:
model_basic = PanelOLS(data.log_flow, data[['log_gdp_o', 'log_gdp_d', 'log_distw']], time_effects=True)
result_basic = model_basic.fit(cov_type='clustered', cluster_entity=True)
result_basic.summary

/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,log_flow,R-squared:,0.3617
Estimator:,PanelOLS,R-squared (Between):,-1.1857
No. Observations:,86431,R-squared (Within):,0.2192
Date:,"Wed, Nov 25 2020",R-squared (Overall):,-0.0111
Time:,19:03:39,Log-likelihood,-9.634e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1.632e+04
Entities:,3779,P-value,0.0000
Avg Obs:,22.871,Distribution:,"F(3,86371)"
Min Obs:,1.0000,,
Max Obs:,57.000,F-statistic (robust):,1654.8


In [6]:
model_modified = PanelOLS(data.log_flow, data[['log_gdp_o', 'log_gdp_d', 'log_distw', 'log_polity_dist']], time_effects=True)
result_modified = model_modified.fit(cov_type='clustered', cluster_entity=True)
result_modified.summary

Dep. Variable:,log_flow,R-squared:,0.3620
Estimator:,PanelOLS,R-squared (Between):,-1.1195
No. Observations:,86431,R-squared (Within):,0.2184
Date:,"Wed, Nov 25 2020",R-squared (Overall):,0.0230
Time:,19:03:40,Log-likelihood,-9.632e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1.225e+04
Entities:,3779,P-value,0.0000
Avg Obs:,22.871,Distribution:,"F(4,86370)"
Min Obs:,1.0000,,
Max Obs:,57.000,F-statistic (robust):,1235.5


In [7]:
model_modified_control = PanelOLS(data.log_flow, data[['log_gdp_o', 'log_gdp_d', 'log_distw', 'log_polity_dist','col_hist']], time_effects=True)
result_modified_control = model_modified_control.fit(cov_type='clustered', cluster_entity=True)
result_modified_control.summary

Dep. Variable:,log_flow,R-squared:,0.3621
Estimator:,PanelOLS,R-squared (Between):,-1.1576
No. Observations:,86431,R-squared (Within):,0.2185
Date:,"Wed, Nov 25 2020",R-squared (Overall):,0.0032
Time:,19:03:41,Log-likelihood,-9.632e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,9806.8
Entities:,3779,P-value,0.0000
Avg Obs:,22.871,Distribution:,"F(5,86369)"
Min Obs:,1.0000,,
Max Obs:,57.000,F-statistic (robust):,988.34


In [8]:
post_89_data = data.query("year >= 1989")

In [9]:
model_basic = PanelOLS(post_89_data.log_flow, post_89_data[['log_gdp_o', 'log_gdp_d', 'log_distw']], time_effects=True)
result_basic = model_basic.fit(cov_type='clustered', cluster_entity=True)
result_basic.summary

/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,log_flow,R-squared:,0.3951
Estimator:,PanelOLS,R-squared (Between):,-0.0377
No. Observations:,40465,R-squared (Within):,0.0523
Date:,"Wed, Nov 25 2020",R-squared (Overall):,0.4004
Time:,19:03:43,Log-likelihood,-4.483e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,8806.6
Entities:,3573,P-value,0.0000
Avg Obs:,11.325,Distribution:,"F(3,40444)"
Min Obs:,1.0000,,
Max Obs:,18.000,F-statistic (robust):,1358.7


In [10]:
model_modified = PanelOLS(post_89_data.log_flow, post_89_data[['log_gdp_o', 'log_gdp_d', 'log_distw', 'log_polity_dist']], time_effects=True)
result_modified = model_modified.fit(cov_type='clustered', cluster_entity=True)
result_modified.summary

Dep. Variable:,log_flow,R-squared:,0.3954
Estimator:,PanelOLS,R-squared (Between):,0.0124
No. Observations:,40465,R-squared (Within):,0.0518
Date:,"Wed, Nov 25 2020",R-squared (Overall):,0.4314
Time:,19:03:44,Log-likelihood,-4.483e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,6612.2
Entities:,3573,P-value,0.0000
Avg Obs:,11.325,Distribution:,"F(4,40443)"
Min Obs:,1.0000,,
Max Obs:,18.000,F-statistic (robust):,1012.6


In [11]:
# World post-1989 exclude China
post_89_noChina_data = data.query("year >= 1989")
post_89_noChina_data = post_89_noChina_data[post_89_noChina_data["iso_o"] != ("CHN" or "USA")]
post_89_noChina_data = post_89_noChina_data[post_89_noChina_data["iso_d"] != ("CHN" or "USA")]

In [12]:
model_basic = PanelOLS(post_89_noChina_data.log_flow, post_89_noChina_data[['log_gdp_o', 'log_gdp_d', 'log_distw']], time_effects=True)
result_basic = model_basic.fit(cov_type='clustered', cluster_entity=True)
result_basic.summary

/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,log_flow,R-squared:,0.3891
Estimator:,PanelOLS,R-squared (Between):,0.0439
No. Observations:,38234,R-squared (Within):,0.0460
Date:,"Wed, Nov 25 2020",R-squared (Overall):,0.4556
Time:,19:03:44,Log-likelihood,-4.29e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,8111.5
Entities:,3425,P-value,0.0000
Avg Obs:,11.163,Distribution:,"F(3,38213)"
Min Obs:,1.0000,,
Max Obs:,18.000,F-statistic (robust):,1298.0


In [13]:
model_modified = PanelOLS(post_89_noChina_data.log_flow, post_89_noChina_data[['log_gdp_o', 'log_gdp_d', 'log_distw', 'log_polity_dist']], time_effects=True)
result_modified = model_modified.fit(cov_type='clustered', cluster_entity=True)
result_modified.summary

Dep. Variable:,log_flow,R-squared:,0.3896
Estimator:,PanelOLS,R-squared (Between):,0.1242
No. Observations:,38234,R-squared (Within):,0.0449
Date:,"Wed, Nov 25 2020",R-squared (Overall):,0.5041
Time:,19:03:45,Log-likelihood,-4.288e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,6098.4
Entities:,3425,P-value,0.0000
Avg Obs:,11.163,Distribution:,"F(4,38212)"
Min Obs:,1.0000,,
Max Obs:,18.000,F-statistic (robust):,962.05


In [14]:
# Dataset of former Soviet countries as the origin from 1960-2006
FORMER_SOVIET_COUNTRIES = {
    "Estonia": "EST",
    "Latvia": "LVA",
    "Lithuania": "LTU",
    "Kazakhstan": "KAZ",
    "Kyrgyzstan": "KGZ",
    "Tajikistan": "TJK",
    "Turkmenistan": "TKM",
    "Uzbekistan": "UZB",
    "Belarus": "BLR",
    "Moldova": "MDA",
    "Ukraine": "UKR",
    "Russia": "RUS",
    "Armenia": "ARM",
    "Azerbaijan": "AZE",
    "Georgia": "GEO"
}
post_soviet_data = data
post_soviet_data = post_soviet_data[post_soviet_data['iso_o'].isin(list(FORMER_SOVIET_COUNTRIES.values()))]
post_soviet_data.head(10)

,,iso_o,iso_d,contig,comlang_off,distw,pop_o,gdp_o,gdpcap_o,iso2_o,pop_d,...,family,polity_o,polity_d,polity_dist,log_gdp_o,log_gdp_d,log_flow,log_distw,log_polity_dist,log_col_hist
tradelink,year,,,,,,,,,,,,,,,,,,,,,
BLR-ARG,1992,BLR,ARG,0,0,12833.472873,10.216000,17022.1800,1666.2275,BY,33.49484,...,NaN,6,7,1,9.742272,12.340513,-0.258322,9.459890,0.693147,0.0
EST-ARG,1992,EST,ARG,0,0,12925.275505,1.533000,3986.6152,2600.5317,EE,33.49484,...,NaN,6,7,1,8.290698,12.340513,NaN,9.467017,0.693147,0.0
RUS-ARG,1992,RUS,ARG,0,0,14195.326754,148.689000,460205.4400,3095.0874,RU,33.49484,...,NaN,5,7,2,13.039429,12.340513,1.597607,9.560739,1.098612,0.0
AZE-AUS,1992,AZE,AUS,0,0,12862.659951,7.382000,4991.3500,676.1515,AZ,17.49500,...,NaN,1,10,9,8.515462,12.681400,0.784793,9.462162,2.302585,0.0
BLR-AUS,1992,BLR,AUS,0,0,14713.924563,10.216000,17022.1800,1666.2275,BY,17.49500,...,NaN,6,10,4,9.742272,12.681400,-0.131073,9.596618,1.609438,0.0
EST-AUS,1992,EST,AUS,0,0,14837.686421,1.533000,3986.6152,2600.5317,EE,17.49500,...,NaN,6,10,4,8.290698,12.681400,NaN,9.604993,1.609438,0.0
RUS-AUS,1992,RUS,AUS,0,0,13424.896701,148.689000,460205.4400,3095.0874,RU,17.49500,...,NaN,5,10,5,13.039429,12.681400,1.151259,9.504941,1.791759,0.0
UKR-AUS,1992,UKR,AUS,0,0,14308.320408,52.150265,73942.3000,1417.8700,UA,17.49500,...,NaN,6,10,4,11.211040,12.681400,NaN,9.568666,1.609438,0.0
BLR-AZE,1992,BLR,AZE,0,0,2152.441933,10.216000,17022.1800,1666.2275,BY,7.38200,...,RUS,6,1,5,9.742272,8.515462,1.551627,7.674823,1.791759,0.0


In [15]:
model_basic = PanelOLS(post_soviet_data.log_flow, post_soviet_data[['log_gdp_o', 'log_gdp_d', 'log_distw']], time_effects=True)
result_basic = model_basic.fit(cov_type='clustered', cluster_entity=True)
result_basic.summary

/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,log_flow,R-squared:,0.3481
Estimator:,PanelOLS,R-squared (Between):,0.6491
No. Observations:,3802,R-squared (Within):,0.0648
Date:,"Wed, Nov 25 2020",R-squared (Overall):,0.7499
Time:,19:03:46,Log-likelihood,-4196.1
Cov. Estimator:,Clustered,,
,,F-statistic:,673.59
Entities:,378,P-value,0.0000
Avg Obs:,10.058,Distribution:,"F(3,3784)"
Min Obs:,1.0000,,
Max Obs:,15.000,F-statistic (robust):,215.74


In [16]:
model_modified = PanelOLS(post_soviet_data.log_flow, post_soviet_data[['log_gdp_o', 'log_gdp_d', 'log_distw', 'log_polity_dist']], time_effects=True)
result_modified = model_modified.fit(cov_type='clustered', cluster_entity=True)
result_modified.summary

Dep. Variable:,log_flow,R-squared:,0.3482
Estimator:,PanelOLS,R-squared (Between):,0.6513
No. Observations:,3802,R-squared (Within):,0.0648
Date:,"Wed, Nov 25 2020",R-squared (Overall):,0.7494
Time:,19:03:46,Log-likelihood,-4196.1
Cov. Estimator:,Clustered,,
,,F-statistic:,505.13
Entities:,378,P-value,0.0000
Avg Obs:,10.058,Distribution:,"F(4,3783)"
Min Obs:,1.0000,,
Max Obs:,15.000,F-statistic (robust):,162.20


In [17]:
post_soviet_89_data = post_soviet_data.query("year >= 1989")
post_soviet_89_data.head()

,,iso_o,iso_d,contig,comlang_off,distw,pop_o,gdp_o,gdpcap_o,iso2_o,pop_d,...,family,polity_o,polity_d,polity_dist,log_gdp_o,log_gdp_d,log_flow,log_distw,log_polity_dist,log_col_hist
tradelink,year,,,,,,,,,,,,,,,,,,,,,
BLR-ARG,1992,BLR,ARG,0,0,12833.472873,10.216,17022.1800,1666.2275,BY,33.49484,...,NaN,6,7,1,9.742272,12.340513,-0.258322,9.459890,0.693147,0.0
EST-ARG,1992,EST,ARG,0,0,12925.275505,1.533,3986.6152,2600.5317,EE,33.49484,...,NaN,6,7,1,8.290698,12.340513,NaN,9.467017,0.693147,0.0
RUS-ARG,1992,RUS,ARG,0,0,14195.326754,148.689,460205.4400,3095.0874,RU,33.49484,...,NaN,5,7,2,13.039429,12.340513,1.597607,9.560739,1.098612,0.0
AZE-AUS,1992,AZE,AUS,0,0,12862.659951,7.382,4991.3500,676.1515,AZ,17.49500,...,NaN,1,10,9,8.515462,12.681400,0.784793,9.462162,2.302585,0.0
BLR-AUS,1992,BLR,AUS,0,0,14713.924563,10.216,17022.1800,1666.2275,BY,17.49500,...,NaN,6,10,4,9.742272,12.681400,-0.131073,9.596618,1.609438,0.0


In [18]:
model_basic = PanelOLS(post_soviet_89_data.log_flow, post_soviet_89_data[['log_gdp_o', 'log_gdp_d', 'log_distw']], time_effects=True)
result_basic = model_basic.fit(cov_type='clustered', cluster_entity=True)
result_basic.summary

/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,log_flow,R-squared:,0.3481
Estimator:,PanelOLS,R-squared (Between):,0.6491
No. Observations:,3802,R-squared (Within):,0.0648
Date:,"Wed, Nov 25 2020",R-squared (Overall):,0.7499
Time:,19:03:47,Log-likelihood,-4196.1
Cov. Estimator:,Clustered,,
,,F-statistic:,673.59
Entities:,378,P-value,0.0000
Avg Obs:,10.058,Distribution:,"F(3,3784)"
Min Obs:,1.0000,,
Max Obs:,15.000,F-statistic (robust):,215.74


In [19]:
model_modified = PanelOLS(post_soviet_89_data.log_flow, post_soviet_89_data[['log_gdp_o', 'log_gdp_d', 'log_distw', 'log_polity_dist']], time_effects=True)
result_modified = model_modified.fit(cov_type='clustered', cluster_entity=True)
result_modified.summary

Dep. Variable:,log_flow,R-squared:,0.3482
Estimator:,PanelOLS,R-squared (Between):,0.6513
No. Observations:,3802,R-squared (Within):,0.0648
Date:,"Wed, Nov 25 2020",R-squared (Overall):,0.7494
Time:,19:03:47,Log-likelihood,-4196.1
Cov. Estimator:,Clustered,,
,,F-statistic:,505.13
Entities:,378,P-value,0.0000
Avg Obs:,10.058,Distribution:,"F(4,3783)"
Min Obs:,1.0000,,
Max Obs:,15.000,F-statistic (robust):,162.20
